## Downloading the required checkpoint:

In [ ]:
import zipfile
import os
import gdown
from pathlib import Path


def download_checkpoints():
    HOME = os.getcwd()
    current = HOME 
    while 'src' not in os.listdir(current):
        current = Path(current).parent
    checkpoint_dir = str(current) + '\\checkpoints'
    url ="https://drive.google.com/file/d/1OzdC7oYtZoQlIEWYPz0Z0OBvzGNMwiTB/view?usp=sharing"
    gdown.download(url, checkpoint_dir + '\\checkpoints.zip', quiet=False, fuzzy=True, use_cookies=False)
    with zipfile.ZipFile(checkpoint_dir + '\\checkpoints.zip', 'r') as zip_ref:
      zip_ref.extractall(checkpoint_dir)
    os.remove(checkpoint_dir + '\\checkpoints.zip')

download_checkpoints()

### Predict Output using the data/test.csv file
If you want to create your own test data set just replace this file with your own test.csv file following the same format.
Results will be shown in results.txt file


In [7]:
import os, sys
from pathlib import Path
HOME = os.getcwd()

current = HOME 
while 'src' not in os.listdir(current):
    current = Path(current).parent

PARENT_DIR = str(current)
DATA_FOLDER = os.path.join(PARENT_DIR,'data')
data_path = os.path.join(DATA_FOLDER, 'filtered.tsv')

sys.path.append(str(current))
print(sys.path)
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
from datasets import load_dataset

from src.models.predict.s2sposttrain import prepare_sample




def predict():
    checkpoint = 'facebook/bart-base'
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    seq2seq_checkpoint = os.path.join(PARENT_DIR, 
                                      'checkpoints', 
                                      's2s', 
                                      'seq2seq_checkpoints', 
                                      'checkpoint-21200')
    model = AutoModelForSeq2SeqLM.from_pretrained(seq2seq_checkpoint).to('cuda')

    test_data = load_dataset("csv", data_files=os.path.join(DATA_FOLDER, 'test.csv'), split='train')
    test_data = test_data.map(lambda b: prepare_sample(b, tokenizer=tokenizer), batched=True).remove_columns(['source', 'target'])

    with open(os.path.join(current, "results.txt"), 'w') as file:
        for i in range(len(test_data)):
            input_ids = test_data[i]['input_ids']
            attention_mask = test_data[i]['attention_mask']

            outputs = model.generate(
                input_ids=torch.tensor(input_ids).unsqueeze(0).to('cuda'),
                attention_mask=torch.tensor(attention_mask).unsqueeze(0).to('cuda'),
                max_length=512,
                num_beams=2,
                early_stopping=True
            )
            generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
            file.write(f'{generated}\n')


if __name__ == '__main__':
    predict()

['c:\\Users\\m4mou\\Desktop\\pmldl\\mywork\\Text-De-toxification\\notebooks', 'c:\\Users\\m4mou\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'c:\\Users\\m4mou\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'c:\\Users\\m4mou\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'c:\\Users\\m4mou\\AppData\\Local\\Programs\\Python\\Python311', '', 'C:\\Users\\m4mou\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\m4mou\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\m4mou\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\m4mou\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\m4mou\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages', 'c:\\Users\\m4mou\\Desktop\\pmldl\\mywork\\Text-De-toxification', 'c:\\Users\\m4mou\\Desktop\\pmldl\\mywork\\Text-De-toxification', 'c:\\Users\\m4mou\\Desktop\\pmldl\\mywork\\Text-De-toxification\\data_analysis', 'c:\\Users\\m4mo

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]